In [1]:
import torch
torch.set_default_dtype(torch.float16)
import torchvision.transforms as transforms
from torchvision import datasets

# loading training data
train_dataset = datasets.MNIST(root='./data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)
#loading test data
test_dataset = datasets.MNIST(root='./data', 
                              train=False, 
                              transform=transforms.ToTensor())

In [2]:
# data preparation
class EmptyClass:
    pass

dev = torch.device('cuda')

images = train_dataset.data
labels = train_dataset.targets

ds = EmptyClass()
ds.N = torch.tensor(len(labels), device=dev)
ds.X = images.view(ds.N, -1)
ds.X = torch.cat((ds.X, torch.ones(ds.N, 1)), dim = 1).to(dev)
ds.d = torch.tensor(ds.X.shape[1], device=dev)
ds.K = torch.tensor(len(train_dataset.classes), device=dev)
ds.Y = labels.to(dev)

In [3]:
ds.x_hat = torch.zeros(ds.K, ds.d).to(dev)
for j in range(ds.N):
    y = labels[j]
    ds.x_hat[y, :] += ds.X[j, :]

ds.alpha = torch.tensor(0.1, device=dev)


In [4]:
inits = EmptyClass()
inits.u0 = torch.randn(ds.K, ds.d).to(dev)
inits.v0 = torch.zeros(ds.N, ds.K, ds.d).to(dev)

inner_params = EmptyClass()
inner_params.n_iter = torch.tensor(10, device=dev)
inner_params.lr = torch.tensor(1e-3, device=dev)

In [8]:
import logistic_splitting

sol = logistic_splitting.parallel_douglas_rachford(0.1, inits, 1, inner_params, ds)

In [7]:
from importlib import reload 
reload(logistic_splitting)

<module 'logistic_splitting' from '/home/zws/Courses/Convex/sc_cvxopt/logistic_splitting.py'>